## Import python FHE compiler frontend

In [20]:
import compyler as cp

import numpy as np

## Create a FHE program object
If the parameter scheme or library is missing,

compiler automatically determines proper scheme or target library.


In [21]:
program = cp.FheProgram(scheme="CKKS", devices=[1])

## Add secret variables: 
```python
add_secret("variable_name", scale)
```

In [22]:
a = program.add_secret("a", 40)

## A function to be a FHE circuit

In [23]:
b = -a + 2
c = a + 0
for i in range(5):
    c = c * (-c + 2)
    b = b * (-c + 2)
d = b.mean() + c.mean()

## Compile the given function into a circuit.
Internally, this call generate HIR (High-level IR) and SIR (Scheme IR) and also apply several optimizations.



In [24]:
program.cppcompile(d)

[2023-11-03 13:18:14.114172] I have received the context:


I have received inputs:
        buffer_bit_length		= 62
        scale_bits			= 40
        logN				= 16
        N				= 65,536
        Number of special primes	= 4
        Number of scales		= 34
        Cache folder			= '/home/kyuyeon/.pyenv/versions/compyler/lib/python3.10/site-packages/liberate/fhe/cache/resources'
        Security bits			= 128
        Quantum security model		= post_quantum
        Security sampling distribution	= uniform
        Number of message bits		= 60
        In total I will be using '1,660' bits out of available maximum '1,661' bits.
        And is it secured?		= True
My RNS primes are [1099510054913, 1099515691009, 1099507695617, 1099516870657, 1099506515969, 1099521458177, 1099503894529, 1099522375681, 1099490000897, 1099523555329, 1099489607681, 1099525128193, 1099486855169, 1099526176769, 1099484889089, 1099529060353, 1099480956929, 1099535220737, 1099469684737, 1099536138241, 1099468767233, 109953

## Create example input data

In [25]:
target_N = 10**5
required_num_ct = round(target_N/2**15)
plain_a = np.linspace(2**(-3), 7*2**(-3), 2**15 * required_num_ct)

## Execute compiled FHE circuit

In [26]:
%%time

result = program.compute({"a" : plain_a})

CPU times: user 1.26 s, sys: 0 ns, total: 1.26 s
Wall time: 1.26 s


## Execute the reference

In [27]:
plain_result = program.plain_compute({"a" : plain_a})

## Compare the results with reference.

In [28]:
print("====="*10)
print(f"|\t\t   HE Result\t\t\t|")
print("-----"*10)
print(f"|\tHE\t:\t{result[0][0].real:19.12e}\t|")
print(f"|\tOrigin\t:\t{plain_result:19.12e}\t|")
print(f"|\tError\t:\t{result[0][0].real - plain_result:19.12e}\t|")
print("====="*10)

|		   HE Result			|
--------------------------------------------------
|	HE	:	 3.594048648665e+00	|
|	Origin	:	 3.594048656672e+00	|
|	Error	:	-8.006795670923e-09	|
